In [ ]:
!git clone https://github.com/chuangchuangtan/FreqNet-DeepfakeDetection.git


Cloning into 'FreqNet-DeepfakeDetection'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 45 (delta 1), reused 1 (delta 1), pack-reused 43 (from 2)
Receiving objects: 100% (45/45), 8.62 MiB | 17.20 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
%cd FreqNet-DeepfakeDetection

/content/FreqNet-DeepfakeDetection


In [ ]:
!pip install -r requirements.txt

  Attempting uninstall: gdown
    Found existing installation: gdown 5.2.0
    Uninstalling gdown-5.2.0:
      Successfully uninstalled gdown-5.2.0


In [ ]:
!pip install gdown==4.7.1

In [ ]:
!pip install -U gdown


  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.1
    Uninstalling gdown-4.7.1:
      Successfully uninstalled gdown-4.7.1


In [ ]:
#!sed -i 's/\r$//' /content/FreqNet-DeepfakeDetection/download_dataset.sh


In [ ]:
#!chmod 777 ./download_dataset.sh

In [ ]:
#!/content/FreqNet-DeepfakeDetection/download_dataset.sh


In [ ]:
%cd /content/

/content


# Deepfake Detection Application - Streamlit

To enable the detections in this notebook, you must ensure that in the 'Runtime' tab, under 'Change runtime type', the mode with a GPU is selected.

To access some examples for testing the App, clone the following repository and use the images and videos from the 'samples' folder:

In [ ]:
!git clone https://github.com/NathFarinha/TCC_DeepFake_Detection_v1.git

Cloning into 'TCC_DeepFake_Detection_v1'...
remote: Enumerating objects: 531, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 531 (delta 49), reused 6 (delta 6), pack-reused 456 (from 1)
Receiving objects: 100% (531/531), 96.73 MiB | 15.32 MiB/s, done.
Resolving deltas: 100% (203/203), done.


**Organizing the Environment for App Execution**

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.1 MB/s eta 0:00:00


In [ ]:
# Cloning 'Video Face Manipulation Detection Through Ensemble of CNNs' repository
!git clone https://github.com/polimi-ispl/icpr2020dfdc
!pip install efficientnet-pytorch
!pip install -U git+https://github.com/albu/albumentations > /dev/null
%cd icpr2020dfdc

Cloning into 'icpr2020dfdc'...
remote: Enumerating objects: 656, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 656 (delta 101), reused 87 (delta 83), pack-reused 537 (from 1)
Receiving objects: 100% (656/656), 99.64 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (341/341), done.
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=d204f5f82c4e794f4558f8158dc55855904846a9a9aea2625261745caa02e5a9
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet-pytorch
  Running command git clone --filter=blob:none --quiet https://github.com/albu/albumentations /tmp/pip-req-build-dage5e33
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

**Streamlit Deepfake Detection App**

In [ ]:
import torch
from torch.utils.model_zoo import load_url
from scipy.special import expit
from PIL import Image
import streamlit as st
import os
import tempfile
import cv2
import numpy as np

from blazeface import FaceExtractor, BlazeFace, VideoReader
from architectures import fornet, weights
from isplutils import utils

# ===== FREQNET IMPORTS =====
# Adjusting sys path to import FreqNet model definition
import sys
sys.path.append("/content/FreqNet-DeepfakeDetection")
from networks.freqnet import freqnet

# Device configuration
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224
frames_per_video = 32

# Initialize BlazeFace (Used by the original paper's models)
facedet = BlazeFace().to(device)
# NOTE: Assuming 'blazeface/blazeface.pth' and 'blazeface/anchors.npy' are available in the environment
facedet.load_weights("blazeface/blazeface.pth")
facedet.load_anchors("blazeface/anchors.npy")
videoreader = VideoReader(verbose=False)
video_read_fn = lambda x: videoreader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn=video_read_fn, facedet=facedet)

# ===== FREQNET FUNCTIONS =====
def load_freqnet_model(model_path):
    """Loads the FreqNet model state dictionary."""
    model = freqnet(num_classes=1)
    model.load_state_dict(torch.load(model_path, map_location=device), strict=True)
    model.to(device)
    model.eval()
    return model

def preprocess_frame_freqnet(frame):
    """Preprocesses a single video frame for FreqNet (resize, normalization)."""
    frame = cv2.resize(frame, (299, 299))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = frame.astype(np.float32) / 255.0
    frame = (frame - 0.5) / 0.5 # Normalize to [-1, 1]
    tensor = torch.from_numpy(frame).permute(2, 0, 1).unsqueeze(0)
    return tensor

def predict_freqnet(file_path, is_video, model):
    """Handles prediction logic specifically for FreqNet (image or video)."""
    scores = []
    if not is_video:
        # Image prediction
        frame = cv2.imread(file_path)
        tensor = preprocess_frame_freqnet(frame).to(device)
        with torch.no_grad():
            score = torch.sigmoid(model(tensor)).item()
        scores.append(score)
    else:
        # Video prediction (sampling every 5th frame)
        cap = cv2.VideoCapture(file_path)
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_count += 1
            if frame_count % 5 == 0:
                tensor = preprocess_frame_freqnet(frame).to(device)
                with torch.no_grad():
                    score = torch.sigmoid(model(tensor)).item()
                scores.append(score)
        cap.release()

    avg_score = np.mean(scores) if scores else 0
    # Thresholding at 0.6 as used in some deepfake contexts
    prediction = "FAKE" if avg_score >= 0.6 else "REAL"
    return prediction, avg_score

# ===== MAIN DETECTION FUNCTION (for ForNet/EfficientNet models) =====
def detect_deep_fake(uploaded_file, selected_model, selected_dataset):
    """Detects deepfakes using the selected ForNet-based model."""

    # Save uploaded file to a temporary directory
    temp_dir = tempfile.mkdtemp()
    temp_file_path = os.path.join(temp_dir, uploaded_file.name)
    with open(temp_file_path, 'wb') as temp_file:
        temp_file.write(uploaded_file.read())

    # Handle FreqNet separately
    if selected_model == "FreqNet":
        model_path = "/content/FreqNet-DeepfakeDetection/4-classes-freqnet-v2.pth"
        is_video = uploaded_file.type.startswith('video')
        # FreqNet has its own prediction logic
        return predict_freqnet(temp_file_path, is_video, load_freqnet_model(model_path))

    # --- ForNet Models (EfficientNet family) ---

    model_url = weights.weight_url['{:s}_{:s}'.format(selected_model, selected_dataset)]
    net = getattr(fornet, selected_model)().eval().to(device)
    # Load weights from URL
    net.load_state_dict(load_url(model_url, map_location=device, check_hash=True))
    transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

    if uploaded_file.type.startswith('image'):
        # Image processing
        im = Image.open(temp_file_path)
        im_faces = face_extractor.process_image(img=im)
        im_face = im_faces['faces'][0] if len(im_faces['faces']) > 0 else None

        if im_face is not None:
            faces_t = torch.stack([transf(image=im_face)['image']])
            with torch.no_grad():
                faces_pred = net(faces_t.to(device)).cpu().numpy().flatten()

            # expit converts logits to probabilities (0-1)
            avg_score = expit(faces_pred.mean())
            prediction = 'FAKE' if avg_score >= 0.6 else 'REAL'
        else:
            return 'Face could not be detected in the image.', 0.0

    elif uploaded_file.type.startswith('video'):
        # Video processing
        vid_faces = face_extractor.process_video(temp_file_path)

        # Stack all detected faces from sampled frames
        faces_t = torch.stack([transf(image=frame['faces'][0])['image']
                               for frame in vid_faces if len(frame['faces'])])

        if faces_t.nelement() == 0:
            return 'No faces detected in video samples.', 0.0

        with torch.no_grad():
            # Get logits from the network
            faces_pred = net(faces_t.to(device)).cpu().numpy().flatten()

        # expit converts logits to probabilities (0-1)
        avg_score = expit(faces_pred.mean())
        prediction = 'FAKE' if avg_score >= 0.6 else 'REAL'
    else:
        return 'Unsupported file type.', 0.0

    return prediction, avg_score

# ===== STREAMLIT APP LAYOUT =====
st.set_page_config(page_title="Deepfake Detection", page_icon="✅", layout="wide")
page = st.sidebar.radio("Select a page", ["Deepfake Detection", "About the Author"])

if page == "Deepfake Detection":
    st.title('Deepfake Detection')
    uploaded_file = st.file_uploader('Upload an image or video', type=['jpg', 'jpeg', 'png', 'mp4'])

    # List of available models
    models = ['EfficientNetB4', 'EfficientNetB4ST', 'EfficientNetAutoAttB4', 'EfficientNetAutoAttB4ST', 'FreqNet']

    if uploaded_file:
        selected_model = st.selectbox(
            'Select model', models
        )
        # Assuming DFDC is the training dataset for the non-FreqNet models
        selected_dataset = 'DFDC'

        if st.button('Detect'):
            with st.spinner('Analyzing file...'):
                prediction, avg_score = detect_deep_fake(uploaded_file, selected_model, selected_dataset)

            if uploaded_file.type.startswith('image'):
                st.image(uploaded_file, caption='Uploaded image', width=500)
            elif uploaded_file.type.startswith('video'):
                st.video(uploaded_file, format='video/mp4')

            st.markdown(f"## **Detection Result: {prediction}**")
            st.write(f'Average Score (Probability of FAKE): {avg_score:.4f}')

elif page == "About the Author":
    st.title('About the Author')
    st.markdown("### Name:")
    st.write("Nathalia Farinha Rodrigues")
    st.markdown("### Position:")
    st.write("Computer Vision Researcher at CPQD")
    st.markdown("### Thesis:")
    st.write("ANALYSIS OF DEEPFAKE DETECTION MODELS USING DEEP LEARNING")
    st.markdown("### Course:")
    st.write("Electrical Engineering")


Writing app.py


In [ ]:
!pip install -q streamlit pyngrok

In [ ]:
from pyngrok import ngrok, conf
import subprocess
import os
import time

# --- ngrok SETUP ---

# NOTE: Paste your valid ngrok authentication token here
# ngrok.set_auth_token("YOUR_NOGROK_AUTH_TOKEN_HERE")
ngrok.set_auth_token("xxx")

# Close old tunnels (if run before)
print("Closing existing ngrok tunnels...")
ngrok.kill()

# Open the tunnel for port 8501 (where Streamlit runs by default)
print("Opening new ngrok tunnel...")
public_url = ngrok.connect(8501)
print("\nStreamlit Public URL:", public_url)

# Run Streamlit in the background
# The shell command sends output logs to /content/logs.txt
print("Starting Streamlit application in background...")
subprocess.Popen(["streamlit", "run", "app.py"], stdout=open("/content/logs.txt", "a"), stderr=subprocess.STDOUT)

# Give Streamlit a moment to start
time.sleep(5)
print("Setup complete. Access the application via the URL above.")



URL pública do Streamlit: NgrokTunnel: "https://95044cc8450d.ngrok-free.app" -> "http://localhost:8501"
